<a href="https://colab.research.google.com/github/aravind-sundaresan/yoda-english-nmt/blob/master/english_to_yoda_nmt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [100]:
from google.colab import drive
drive.mount('/drive')

Drive already mounted at /drive; to attempt to forcibly remount, call drive.mount("/drive", force_remount=True).


**Data Preparation**

In [0]:
# Create 2 dictionaries each (word-index and index-word) for the source and target languages

with open('/drive/My Drive/data/english_vocabulary.txt', 'r') as f:
  words = f.readlines()

english_index_to_word_dict = dict([(index, word.rstrip('\n')) for index, word in enumerate(words)])
english_word_to_index_dict = dict([(word.rstrip('\n'), index) for index, word in enumerate(words)])
english_vocab_length = len(english_word_to_index_dict)


with open('/drive/My Drive/data/yoda_vocabulary.txt', 'r') as f:
  words = f.readlines()

yoda_index_to_word_dict = dict([(index, word.rstrip('\n')) for index, word in enumerate(words)])
yoda_word_to_index_dict = dict([(word.rstrip('\n'), index) for index, word in enumerate(words)])
yoda_vocab_length = len(yoda_word_to_index_dict)


# Limiting the lengths of the sequences (in terms of number of words) in both the source and target languages 
# For source language, max. length = 15 (97% of the sentences have length <= 15)
# For target language, max. length = 20 (97% of the sentence have length <= 20)

max_length_source, max_length_target = 15, 20 

english_sentences, yoda_english_sentences = [], []
with open('/drive/My Drive/data/english_sentences.txt', 'r') as fp: 
  line = fp.readline()
  while line:
    english_sentences.append(line.rstrip("\n"))
    line = fp.readline()

# Adding the start and end tokens to the target sentences
with open('/drive/My Drive/data/yoda_english_sentences.txt', 'r') as fp:
  line = fp.readline()
  while line:
    yoda_english_sentences.append("1 " + line.rstrip("\n") + " 2")
    line = fp.readline() 


In [102]:
yoda_english_sentences[:5]

['1 7690 5 68 22 5 36 34 2',
 '1 17935 5 13647 930 930 987 68 22 5 36 34 12 5 14 4 2',
 '1 25 1729 7 89627 66587 362 55 23813 204 9 2',
 '1 622 5 273 27 5 36 34 13 4 2',
 '1 1326 5 491 492 92 1205 5 36 34 2']

In [103]:
english_sentences[:5]

['59 19 2359 27',
 '18506 882 882 2558 59 19 12022 27',
 '65 59177 428 14 2269 10 96825 72439 550 7',
 '422 23 161 27',
 '843 593 161 1256 39 27']